<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_2/01_simple_melody_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Melody RNNを使ってみよう！
「Melody RNN」を使い、曲をを生成してみましょう。

## ライブラリのインストール
Magentaと共に、音楽生成用のライブラリpyFluidSynth、MIDIデータを処理するためのpretty_midiなどをインストールします。

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## NoteSequenceで音を鳴らす

今回は、Section1で作成した「きらきら星」のNoteSequenceをベースに曲を生成します。  
以下のコードは、前回のコードに曲のテンポおよび所要時間を指定する行を加えたものです。  
`qpm`のオプションにより、1分間における四分音符の数を指定することができます。


In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

kira2 = music_pb2.NoteSequence()  # NoteSequence

# notesにnoteを追加
kira2.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
kira2.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
kira2.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
kira2.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
kira2.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
kira2.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
kira2.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
kira2.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
kira2.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
kira2.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

kira2.total_time = 6.4  # 所要時間
kira2.tempos.add(qpm=75);  # 曲のテンポを指定

note_seq.plot_sequence(kira2)  # NoteSequenceの可視化
note_seq.play_sequence(kira2, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## MelodyRNN

MelodyRNNは、RNNの一種「LSTM」をベースにした曲を生成するモデルです。  
これにより、直近のnoteの並びから次のnoteが予測されます。  
これを繰り返すことで、曲が生成されることになります。  
Magentaには数千曲のMIDIデータを使って学習した学習済みのモデルが含まれており、この学習済みモデルはそのまま作曲に使用することができます。  

学習済みのモデルは Bundleファイル（.magファイル）に保存されています。  
以下のコードは、学習済みモデル「basic_rnn.mag」を読み込み、曲の生成器を設定しています。

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# モデルの初期化
note_seq.notebook_utils.download_bundle("basic_rnn.mag", "/models/")  # Bundle（.magファイル）をダウンロード
bundle = sequence_generator_bundle.read_bundle_file("/models/basic_rnn.mag")  # Bundleの読み込み
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map["basic_rnn"](checkpoint=None, bundle=bundle)  # 生成器の設定
melody_rnn.initialize()  # 初期化

## 曲の生成

各設定を行った上で、生成器により曲を生成します。  
`temperature`を変更することで、曲の「ランダム度合い」を調整することができます。

In [ ]:
from note_seq.protobuf import generator_pb2

base_sequence = kira2  # ベースになるNoteSeqence
total_time = 36 # 曲の長さ（秒）
temperature = 1.2 # 曲の「ランダム度合い」を決める定数

base_end_time = max(note.end_time for note in base_sequence.notes)  #ベース曲の終了時刻

# 生成器に関する設定
generator_options = generator_pb2.GeneratorOptions()  # 生成器のオプション
generator_options.args["temperature"].float_value = temperature  # ランダム度合い
generator_options.generate_sections.add(
    start_time=base_end_time,  # 作曲開始時刻
    end_time=total_time)  # 作曲終了時刻

# 曲の生成
gen_seq = melody_rnn.generate(base_sequence, generator_options)

note_seq.plot_sequence(gen_seq)  # NoteSequenceの可視化
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## MIDIファイルの保存とダウンロード
`NoteSequence`をMIDIデータに変換し、保存してダウンロードします。


In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(gen_seq, "simple_melody_rnn.mid")  #MIDI　データに変換し保存
files.download("simple_melody_rnn.mid")  # ダウンロード